In [1]:
import random

import numpy as np
from chainercv.datasets import VOCBboxDataset
from YOLOv3.lib.datasets.yolo_voc_dataset import YOLOVOCDataset

/opt/conda/lib/python3.7/site-packages/chainer/_environment_check.py:73: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy) version 6.0.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy>=6.5.0,<7.0.0'

See the following page for more details:
  https://docs-cupy.chainer.org/en/latest/install.html
--------------------------------------------------------------------------------

  requirement=requirement, help=help))


In [ ]:
train = VOCBboxDataset(split='train')
train = YOLOVOCDataset(train, classifier=False, jitter=0.3, hue=0.1, sat=1.5, val=1.5)
train_iter = chainer.iterators.SerialIterator(train, args.batchsize)

From: http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
To: /home/kitamura/.chainer/dataset/_dl_cache/92029705a99338d0932803388148a725
  %   Total    Recv       Speed  Time left
 36 1907MiB  688MiB    307KiB/s    1:07:50